In [1]:
#!conda install pandas numpy statsmodels openpyxl

In [2]:
from itertools import product
from typing import List, Tuple

import pandas as pd
import numpy as np

import statsmodels.api as sm
from statsmodels.formula.api import ols


def gen_plan(*factors: Tuple[str, List], res_str: str, replications=3, save_path: str):
    assert(len(factors) > 0)
    assert(replications > 1)
    
    factor_names, factor_levels = zip(*factors)
    cp = np.array(list(product(*factor_levels, range(replications))))
    df_dict = {}
    df_dict[res_str] = ""
    for i in range(len(factor_names)):
        df_dict[factor_names[i]] = cp[:, i]
    df = pd.DataFrame(df_dict)
    df.to_excel(save_path, index=False)

def run_plan(save_path: str, sheet_name=0) -> sm.stats.anova_lm:
    df = pd.read_excel(save_path, sheet_name=sheet_name)
    key_it = iter(df.keys())
    modelString = f"{next(key_it)} ~ C({next(key_it)})"
    for key in key_it:
        modelString += f"*C({key})"
    return ols(modelString, df).fit()

# example usage:
# gen_plan(('Processors', ['A', 'B']), ('Workloads', ['C', 'D']), res_str="Code_Size", replications=2, save_path='test.xlsx')
# fill in experiments results in spreadsheet
# model = run_plan('test.xlsx', 0)
# model.summary()
# sm.stats.anova_lm(model)

# 2^k Factorial
### "executorMemory" fixed at 8Gi for all experiments

## FASHIONMNIST, FASHIONMNISTCNN

In [3]:
model_fashion_2k_processor_util = run_plan('fashion-2k.xlsx', 'AvgProcessorUtil')
model_fashion_2k_accuracy = run_plan('fashion-2k.xlsx', 'AvgAccuracy')
model_fashion_2k_train_time = run_plan('fashion-2k.xlsx', 'AvgTrainTime')

In [4]:
sm.stats.anova_lm(model_fashion_2k_processor_util)

,df,sum_sq,mean_sq,F,PR(>F)
C(dataParallelism),1.0,19.859178,19.859178,15.828473,1.079753e-03
C(executorCores),1.0,3693.421918,3693.421918,2943.789068,1.435550e-19
C(batchSize),1.0,27.046174,27.046174,21.556766,2.707718e-04
C(maxEpoch),1.0,0.004680,0.004680,0.003730,9.520546e-01
C(dataParallelism):C(executorCores),1.0,1.365791,1.365791,1.088584,3.122938e-01
C(dataParallelism):C(batchSize),1.0,2.498171,2.498171,1.991132,1.773711e-01
C(executorCores):C(batchSize),1.0,18.955864,18.955864,15.108500,1.309238e-03
C(dataParallelism):C(maxEpoch),1.0,0.010549,0.010549,0.008408,9.280795e-01
C(executorCores):C(maxEpoch),1.0,0.004680,0.004680,0.003730,9.520546e-01
C(batchSize):C(maxEpoch),1.0,0.792855,0.792855,0.631934,4.382806e-01


In [5]:
model_fashion_2k_processor_util.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       AvgProcessorUtil   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     200.4
Date:                Sun, 24 Oct 2021   Prob (F-statistic):           2.66e-15
Time:                        19:59:19   Log-Likelihood:                -37.945
No. Observations:                  32   AIC:                             107.9
Df Residuals:                      16   BIC:                             131.3
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================================================
                                                                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                 26.4495      0.792     33.394      0.000      24.770      28.129
C(dataParallelism)[T.4]                                                                    0.1645      1.120      0.147      0.885      -2.210       2.539
C(executorCores)[T.3500m]                                                                 23.2165      1.120     20.727      0.000      20.842      25.591
C(batchSize)[T.256]                                                                        0.0165      1.120      0.015      0.988      -2.358       2.391
C(maxEpoch)[T.5]                                                                          -0.0195      1.120     -0.017      0.986      -2.394       2.355
C(dataParallelism)[T.4]:C(executorCores)[T.3500m]                                         -1.7005      1.584     -1.073      0.299      -5.059       1.658
C(dataParallelism)[T.4]:C(batchSize)[T.256]                                               -2.5325      1.584     -1.599      0.129      -5.891       0.826
C(executorCores)[T.3500m]:C(batchSize)[T.256]                                             -2.6330      1.584     -1.662      0.116      -5.991       0.725
C(dataParallelism)[T.4]:C(maxEpoch)[T.5]                                                  -1.9585      1.584     -1.236      0.234      -5.317       1.400
C(executorCores)[T.3500m]:C(maxEpoch)[T.5]                                                 0.0230      1.584      0.015      0.989      -3.335       3.381
C(batchSize)[T.256]:C(maxEpoch)[T.5]                                                       0.0635      1.584      0.040      0.969      -3.295       3.422
C(dataParallelism)[T.4]:C(executorCores)[T.3500m]:C(batchSize)[T.256]                      1.6510      2.240      0.737      0.472      -3.098       6.400
C(dataParallelism)[T.4]:C(executorCores)[T.3500m]:C(maxEpoch)[T.5]                         2.5930      2.240      1.157      0.264      -2.156       7.342
C(dataParallelism)[T.4]:C(batchSize)[T.256]:C(maxEpoch)[T.5]                               3.6745      2.240      1.640      0.120      -1.075       8.424
C(executorCores)[T.3500m]:C(batchSize)[T.256]:C(maxEpoch)[T.5]                            -0.0465      2.240     -0.021      0.984      -4.796       4.703
C(dataParallelism)[T.4]:C(executorCores)[T.3500m]:C(batchSize)[T.256]:C(maxEpoch)[T.5]    -4.9915      3.168     -1.576      0.135     -11.708       1.725
==============================================================================
Omnibus:                        1.949   Durbin-Watson:     

# ---

In [6]:
sm.stats.anova_lm(model_fashion_2k_accuracy)

,df,sum_sq,mean_sq,F,PR(>F)
C(dataParallelism),1.0,1.241415e+02,1.241415e+02,1.828161e+27,6.759949e-210
C(executorCores),1.0,8.077936e-28,8.077936e-28,1.189591e-02,9.145044e-01
C(batchSize),1.0,1.386113e+00,1.386113e+00,2.041248e+25,2.798286e-194
C(maxEpoch),1.0,4.437221e+00,4.437221e+00,6.534439e+25,2.537417e-198
C(dataParallelism):C(executorCores),1.0,1.817536e-27,1.817536e-27,2.676580e-02,8.720927e-01
C(dataParallelism):C(batchSize),1.0,1.522513e+00,1.522513e+00,2.242117e+25,1.320666e-194
C(executorCores):C(batchSize),1.0,5.048710e-29,5.048710e-29,7.434944e-04,9.785839e-01
C(dataParallelism):C(maxEpoch),1.0,4.970045e-01,4.970045e-01,7.319099e+24,1.024230e-190
C(executorCores):C(maxEpoch),1.0,1.029476e-27,1.029476e-27,1.516050e-02,9.035392e-01
C(batchSize):C(maxEpoch),1.0,1.428050e-02,1.428050e-02,2.103007e+23,2.204615e-178


In [7]:
sm.stats.anova_lm(model_fashion_2k_train_time)

,df,sum_sq,mean_sq,F,PR(>F)
C(dataParallelism),1.0,4.677409e+10,4.677409e+10,14815.444678,3.604119e-25
C(executorCores),1.0,1.802433e+10,1.802433e+10,5709.110115,7.316912e-22
C(batchSize),1.0,5.867128e+08,5.867128e+08,185.838161,3.176220e-10
C(maxEpoch),1.0,7.906012e+09,7.906012e+09,2504.187248,5.197865e-19
C(dataParallelism):C(executorCores),1.0,4.735585e+09,4.735585e+09,1499.971336,3.037928e-17
C(dataParallelism):C(batchSize),1.0,2.967698e+06,2.967698e+06,0.940003,3.467062e-01
C(executorCores):C(batchSize),1.0,1.322009e+08,1.322009e+08,41.873919,7.720474e-06
C(dataParallelism):C(maxEpoch),1.0,2.263073e+09,2.263073e+09,716.816260,1.024128e-14
C(executorCores):C(maxEpoch),1.0,7.801513e+08,7.801513e+08,247.108798,3.776148e-11
C(batchSize):C(maxEpoch),1.0,1.007466e+07,1.007466e+07,3.191096,9.300180e-02


## CIFAR10, CIFAR10CNN

In [8]:
model_cifar10_2k_processor_util = run_plan('cifar10-2k.xlsx', 'AvgProcessorUtil')
model_cifar10_2k_accuracy = run_plan('cifar10-2k.xlsx', 'AvgAccuracy')
model_cifar10_2k_train_time = run_plan('cifar10-2k.xlsx', 'AvgTrainTime')

In [9]:
sm.stats.anova_lm(model_cifar10_2k_processor_util)

,df,sum_sq,mean_sq,F,PR(>F)
C(dataParallelism),1.0,16.940565,16.940565,21.680473,2.634734e-04
C(executorCores),1.0,4167.000518,4167.000518,5332.911877,1.260414e-21
C(batchSize),1.0,17.244596,17.244596,22.069571,2.419248e-04
C(maxEpoch),1.0,0.250809,0.250809,0.320984,5.788781e-01
C(dataParallelism):C(executorCores),1.0,2.702231,2.702231,3.458306,8.142194e-02
C(dataParallelism):C(batchSize),1.0,5.879878,5.879878,7.525046,1.443494e-02
C(executorCores):C(batchSize),1.0,4.359366,4.359366,5.579101,3.118520e-02
C(dataParallelism):C(maxEpoch),1.0,0.183467,0.183467,0.234800,6.345541e-01
C(executorCores):C(maxEpoch),1.0,0.001785,0.001785,0.002284,9.624702e-01
C(batchSize):C(maxEpoch),1.0,0.010476,0.010476,0.013408,9.092592e-01


In [10]:
model_cifar10_2k_processor_util.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       AvgProcessorUtil   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     359.7
Date:                Sun, 24 Oct 2021   Prob (F-statistic):           2.55e-17
Time:                        19:59:19   Log-Likelihood:                -30.368
No. Observations:                  32   AIC:                             92.74
Df Residuals:                      16   BIC:                             116.2
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================================================
                                                                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                 26.5000      0.625     42.397      0.000      25.175      27.825
C(dataParallelism)[T.4]                                                                    0.0140      0.884      0.016      0.988      -1.860       1.888
C(executorCores)[T.3500m]                                                                 24.0000      0.884     27.151      0.000      22.126      25.874
C(batchSize)[T.256]                                                                    -1.895e-14      0.884  -2.14e-14      1.000      -1.874       1.874
C(maxEpoch)[T.5]                                                                           0.0100      0.884      0.011      0.991      -1.864       1.884
C(dataParallelism)[T.4]:C(executorCores)[T.3500m]                                         -1.4660      1.250     -1.173      0.258      -4.116       1.184
C(dataParallelism)[T.4]:C(batchSize)[T.256]                                               -2.1500      1.250     -1.720      0.105      -4.800       0.500
C(executorCores)[T.3500m]:C(batchSize)[T.256]                                             -1.2335      1.250     -0.987      0.338      -3.884       1.417
C(dataParallelism)[T.4]:C(maxEpoch)[T.5]                                                  -0.3260      1.250     -0.261      0.798      -2.976       2.324
C(executorCores)[T.3500m]:C(maxEpoch)[T.5]                                                 0.0195      1.250      0.016      0.988      -2.631       2.670
C(batchSize)[T.256]:C(maxEpoch)[T.5]                                                      -0.0100      1.250     -0.008      0.994      -2.660       2.640
C(dataParallelism)[T.4]:C(executorCores)[T.3500m]:C(batchSize)[T.256]                      0.7495      1.768      0.424      0.677      -2.998       4.497
C(dataParallelism)[T.4]:C(executorCores)[T.3500m]:C(maxEpoch)[T.5]                         1.1365      1.768      0.643      0.529      -2.611       4.884
C(dataParallelism)[T.4]:C(batchSize)[T.256]:C(maxEpoch)[T.5]                               1.4000      1.768      0.792      0.440      -2.348       5.148
C(executorCores)[T.3500m]:C(batchSize)[T.256]:C(maxEpoch)[T.5]                             0.0435      1.768      0.025      0.981      -3.704       3.791
C(dataParallelism)[T.4]:C(executorCores)[T.3500m]:C(batchSize)[T.256]:C(maxEpoch)[T.5]    -2.5575      2.500     -1.023      0.322      -7.858       2.743
==============================================================================
Omnibus:                       11.030   Durbin-Watson:     

# ---

In [11]:
sm.stats.anova_lm(model_cifar10_2k_accuracy)

,df,sum_sq,mean_sq,F,PR(>F)
C(dataParallelism),1.0,1.279371e+03,1.279371e+03,4.893178e+28,2.566437e-221
C(executorCores),1.0,3.155444e-28,3.155444e-28,1.206855e-02,9.138889e-01
C(batchSize),1.0,8.972641e+01,8.972641e+01,3.431744e+27,4.384692e-212
C(maxEpoch),1.0,1.177499e+02,1.177499e+02,4.503551e+27,4.984461e-213
C(dataParallelism):C(executorCores),1.0,2.473868e-27,2.473868e-27,9.461743e-02,7.623550e-01
C(dataParallelism):C(batchSize),1.0,1.294762e+02,1.294762e+02,4.952046e+27,2.332281e-213
C(executorCores):C(batchSize),1.0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
C(dataParallelism):C(maxEpoch),1.0,2.464020e-01,2.464020e-01,9.424078e+24,1.355650e-191
C(executorCores):C(maxEpoch),1.0,5.129568e-28,5.129568e-28,1.961894e-02,8.903547e-01
C(batchSize):C(maxEpoch),1.0,2.450000e-03,2.450000e-03,9.370456e+22,1.418968e-175


In [12]:
sm.stats.anova_lm(model_cifar10_2k_train_time)

,df,sum_sq,mean_sq,F,PR(>F)
C(dataParallelism),1.0,2.298950e+12,2.298950e+12,23105.583160,1.032884e-26
C(executorCores),1.0,8.249171e+11,8.249171e+11,8290.825188,3.723488e-23
C(batchSize),1.0,3.729589e+06,3.729589e+06,0.037484,8.489190e-01
C(maxEpoch),1.0,2.881030e+11,2.881030e+11,2895.577187,1.637176e-19
C(dataParallelism):C(executorCores),1.0,2.780529e+11,2.780529e+11,2794.569394,2.171729e-19
C(dataParallelism):C(batchSize),1.0,1.092747e+08,1.092747e+08,1.098265,3.102208e-01
C(executorCores):C(batchSize),1.0,8.760180e+07,8.760180e+07,0.880441,3.620305e-01
C(dataParallelism):C(maxEpoch),1.0,9.840455e+10,9.840455e+10,989.014390,8.161604e-16
C(executorCores):C(maxEpoch),1.0,3.384857e+10,3.384857e+10,340.194891,3.322069e-12
C(batchSize):C(maxEpoch),1.0,2.817115e+07,2.817115e+07,0.283134,6.019657e-01


# 2-Factor Full Factorial
### "executorMemory" fixed at 8Gi for all experiments
### "dataParallelism" fixed at 4 for all experiments
### "maxEpoch" fixed at 4 for all experiments

## FASHIONMNIST, FASHIONMNISTCNN

In [13]:
model_fashion_full_processor_util = run_plan('fashion-full.xlsx', 'AvgProcessorUtil')
model_fashion_full_accuracy = run_plan('fashion-full.xlsx', 'AvgAccuracy')
model_fashion_full_train_time = run_plan('fashion-full.xlsx', 'AvgTrainTime')

In [14]:
sm.stats.anova_lm(model_fashion_full_processor_util)

,df,sum_sq,mean_sq,F,PR(>F)
C(executorCores),2.0,3620.011449,1810.005724,311.735779,6.728009e-18
C(batchSize),3.0,28.444028,9.481343,1.632964,2.080621e-01
C(executorCores):C(batchSize),6.0,22.067002,3.677834,0.633430,7.022450e-01
Residual,24.0,139.349219,5.806217,NaN,NaN


In [15]:
model_fashion_full_processor_util.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       AvgProcessorUtil   R-squared:                       0.963
Model:                            OLS   Adj. R-squared:                  0.947
Method:                 Least Squares   F-statistic:                     57.47
Date:                Sun, 24 Oct 2021   Prob (F-statistic):           1.68e-14
Time:                        19:59:20   Log-Likelihood:                -75.444
No. Observations:                  36   AIC:                             174.9
Df Residuals:                      24   BIC:                             193.9
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                        24.6917      1.391     17.749      0.000      21.820      27.563
C(executorCores)[T.2000m]                        21.6917      1.967     11.025      0.000      17.631      25.752
C(executorCores)[T.3500m]                        21.5080      1.967     10.932      0.000      17.447      25.569
C(batchSize)[T.64]                                0.3497      1.967      0.178      0.860      -3.711       4.410
C(batchSize)[T.128]                              -1.9917      1.967     -1.012      0.321      -6.052       2.069
C(batchSize)[T.256]                              -0.1167      1.967     -0.059      0.953      -4.177       3.944
C(executorCores)[T.2000m]:C(batchSize)[T.64]     -3.1163      2.782     -1.120      0.274      -8.859       2.626
C(executorCores)[T.3500m]:C(batchSize)[T.64]      1.8340      2.782      0.659      0.516      -3.909       7.577
C(executorCores)[T.2000m]:C(batchSize)[T.128]    -0.8003      2.782     -0.288      0.776      -6.543       4.942
C(executorCores)[T.3500m]:C(batchSize)[T.128]     0.1670      2.782      0.060      0.953      -5.576       5.910
C(executorCores)[T.2000m]:C(batchSize)[T.256]    -2.0587      2.782     -0.740      0.467      -7.801       3.684
C(executorCores)[T.3500m]:C(batchSize)[T.256]     0.7837      2.782      0.282      0.781      -4.959       6.526
==============================================================================
Omnibus:                        0.273   Durbin-Watson:                   3.100
Prob(Omnibus):                  0.872   Jarque-Bera (JB):                0.307
Skew:                          -0.185   Prob(JB):                        0.858
Kurtosis:                       2.741   Cond. No.                         17.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# ---

In [16]:
sm.stats.anova_lm(model_fashion_full_accuracy)

,df,sum_sq,mean_sq,F,PR(>F)
C(executorCores),2.0,3.685558e-27,1.842779e-27,6.186441e-02,9.401597e-01
C(batchSize),3.0,2.616355e+01,8.721183e+00,2.927811e+26,6.979365e-307
C(executorCores):C(batchSize),6.0,4.723699e-27,7.872832e-28,2.643008e-02,9.999025e-01
Residual,24.0,7.148973e-25,2.978739e-26,NaN,NaN


In [17]:
sm.stats.anova_lm(model_fashion_full_train_time)

,df,sum_sq,mean_sq,F,PR(>F)
C(executorCores),2.0,5.093723e+09,2.546861e+09,16310.952061,2.492244e-38
C(batchSize),3.0,1.000924e+09,3.336413e+08,2136.750261,2.917843e-29
C(executorCores):C(batchSize),6.0,1.760206e+08,2.933677e+07,187.882490,5.894695e-19
Residual,24.0,3.747462e+06,1.561442e+05,NaN,NaN


## CIFAR10, CIFAR10CNN

In [18]:
model_cifar10_full_processor_util = run_plan('cifar10-full.xlsx', 'AvgProcessorUtil')
model_cifar10_full_accuracy = run_plan('cifar10-full.xlsx', 'AvgAccuracy')
model_cifar10_full_train_time = run_plan('cifar10-full.xlsx', 'AvgTrainTime')

In [19]:
sm.stats.anova_lm(model_cifar10_full_processor_util)

,df,sum_sq,mean_sq,F,PR(>F)
C(executorCores),2.0,3702.217611,1851.108806,603.133865,3.037661e-21
C(batchSize),3.0,25.091479,8.363826,2.725127,6.649304e-02
C(executorCores):C(batchSize),6.0,28.850295,4.808383,1.566682,2.000053e-01
Residual,24.0,73.659620,3.069151,NaN,NaN


In [20]:
model_cifar10_full_processor_util.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       AvgProcessorUtil   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.972
Method:                 Least Squares   F-statistic:                     111.3
Date:                Sun, 24 Oct 2021   Prob (F-statistic):           8.07e-18
Time:                        19:59:20   Log-Likelihood:                -63.969
No. Observations:                  36   AIC:                             151.9
Df Residuals:                      24   BIC:                             170.9
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                        26.4750      1.011     26.175      0.000      24.387      28.563
C(executorCores)[T.2000m]                        21.0500      1.430     14.716      0.000      18.098      24.002
C(executorCores)[T.3500m]                        19.6833      1.430     13.761      0.000      16.731      22.636
C(batchSize)[T.64]                               -2.1833      1.430     -1.526      0.140      -5.136       0.769
C(batchSize)[T.128]                              -0.2837      1.430     -0.198      0.844      -3.236       2.669
C(batchSize)[T.256]                              -3.3500      1.430     -2.342      0.028      -6.302      -0.398
C(executorCores)[T.2000m]:C(batchSize)[T.64]      1.9000      2.023      0.939      0.357      -2.275       6.075
C(executorCores)[T.3500m]:C(batchSize)[T.64]      3.0330      2.023      1.499      0.147      -1.142       7.208
C(executorCores)[T.2000m]:C(batchSize)[T.128]    -1.4833      2.023     -0.733      0.470      -5.658       2.692
C(executorCores)[T.3500m]:C(batchSize)[T.128]     1.9087      2.023      0.944      0.355      -2.266       6.084
C(executorCores)[T.2000m]:C(batchSize)[T.256]     2.6830      2.023      1.326      0.197      -1.492       6.858
C(executorCores)[T.3500m]:C(batchSize)[T.256]     1.0663      2.023      0.527      0.603      -3.109       5.241
==============================================================================
Omnibus:                        2.784   Durbin-Watson:                   2.571
Prob(Omnibus):                  0.249   Jarque-Bera (JB):                2.018
Skew:                           0.578   Prob(JB):                        0.365
Kurtosis:                       3.086   Cond. No.                         17.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# ---

In [21]:
sm.stats.anova_lm(model_cifar10_full_accuracy)

,df,sum_sq,mean_sq,F,PR(>F)
C(executorCores),2.0,1.009742e-28,5.048710e-29,5.852231e-03,0.994166
C(batchSize),3.0,2.753125e+02,9.177083e+01,1.063765e+28,0.000000
C(executorCores):C(batchSize),6.0,8.361926e-28,1.393654e-28,1.615460e-02,0.999977
Residual,24.0,2.070476e-25,8.626983e-27,NaN,NaN


In [22]:
sm.stats.anova_lm(model_cifar10_full_train_time)

,df,sum_sq,mean_sq,F,PR(>F)
C(executorCores),2.0,1.304330e+11,6.521648e+10,7158.868486,4.822962e-34
C(batchSize),3.0,2.781781e+09,9.272604e+08,101.786167,8.722440e-14
C(executorCores):C(batchSize),6.0,8.233005e+08,1.372167e+08,15.062400,4.312979e-07
Residual,24.0,2.186373e+08,9.109886e+06,NaN,NaN
